In [ ]:
%load_ext autoreload
%autoreload 2

from diveslowlearnfast.train import StatsDB
from diveslowlearnfast.train.stats import get_value, get_tuple, get_column, get_dict, get_df

In [ ]:
stats = StatsDB('../run26/stats.db')

run22_id, run25_id = stats.execute_query('SELECT DISTINCT(run_id) FROM stats', extractor=get_tuple)

run22_id, run25_id

In [ ]:
start_epoch = stats.execute_query('SELECT MIN(epoch) FROM stats WHERE run_id = ?', run25_id, extractor=get_value)

start_epoch

In [ ]:
result = stats.get_lowest_percentile(
    start_epoch - 10,
    run22_id,
    'train',
)
columns = ['video_id', 'gt', 'percentage']
result_dict = get_dict(result, columns)
result_df = get_df(result, columns)
video_ids = result_dict['video_id']

len(video_ids)

In [ ]:
print('Pre EAL finetuning accuracy:')
result_df['percentage'].mean()

In [ ]:
result = stats.execute_query(
f"""SELECT video_id, gt, (correct_n / n) as acc FROM(
            SELECT
                video_id,
                gt,
                epoch,
                CAST(SUM(CASE WHEN pred = gt THEN 1 ELSE 0 END) as REAL) as correct_n,
                CAST(COUNT(*) as REAL) as n
            FROM stats
            WHERE epoch > ?
            AND video_id IN ({','.join(['?'] * len(video_ids))})
            AND run_id = ?
            AND split = ?
            GROUP BY video_id, gt
        ) ORDER BY acc
        """,
    start_epoch,
    *video_ids,
    run25_id,
    'train'
)

In [ ]:
run25_id

In [ ]:
len(stats.execute_query('SELECT DISTINCT(video_id) FROM stats WHERE epoch < 40 AND split = "train"')), len(stats.execute_query('SELECT DISTINCT(video_id) FROM stats WHERE epoch = 145 AND split = "train"'))

In [ ]:
stats.execute_query('SELECT DISTINCT(video_id) FROM stats WHERE epoch = 147 AND split = "train" LIMIT 1')

In [ ]:
stats.execute_query('SELECT DISTINCT(video_id) FROM stats WHERE epoch = 148 AND split = "train" LIMIT 1')

In [ ]:
stats.execute_query('SELECT DISTINCT(video_id) FROM stats WHERE epoch = 149 AND split = "train" LIMIT 1')

In [ ]:
stats.execute_query('SELECT MAX(id) FROM stats WHERE split = "train"')